In [2]:
import numpy as np
import pandas as pd
import math as m
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
import statsmodels.api as sm
from scipy.stats import pearsonr
from sklearn.decomposition  import PCA
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform
from statsmodels.tsa.seasonal import seasonal_decompose
from stldecompose import decompose, forecast
from stldecompose.forecast_funcs import (naive, drift, mean, seasonal_naive)
from statsmodels.stats.anova import anova_lm
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>"))

## Notebook: series de tiempo de riñas en Bogotá 2014 -2019
>#### Autores: Jorge Victorino - Miguel Barrero
>#### Conjunto de datos: Nuse 2014-2019

#### 1. Carga de los datos  

In [3]:
# lectura de los datos
nuse1 = pd.read_csv('../dataNuse/nuse_110220.csv', sep=',', encoding='utf-8', error_bad_lines=False)

/home/mike/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (9,11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


#### 2. Filtrado de los datos.
> Los datos que se tienen en cuenta en un filtrado inicial son aquellos que tienen códigos de UPZ válidos.

##### 2.1 Función de filtrado
> Parámetros: (df) : datataframe original  de Nuse

In [4]:
# agregación de columnas y filtrado de los datos nulos
def data_crime(df):
    # filtrado de los datos
    dfo = df[['LONGITUD', 'LATITUD', 'LOCALIDAD', 'COD_UPZ', 'UPZ', 
              'FECHA','MES','ANIO']].sort_values(by=['COD_UPZ'])
    dfo = dfo[((dfo['COD_UPZ'] !=  'ND')  & (dfo['COD_UPZ']  != 'UPZ999'))]
    dfo.reset_index(inplace=True)
    dfo = dfo.rename(columns={'index':'INDEX_TIME'})
    anio = pd.to_datetime('2013-12-30')
    # generando nuevas columnas
    dfo['JORN'] = pd.to_datetime(dfo['FECHA']).dt.hour//6
    dfo['DIA'] = pd.to_datetime(dfo['FECHA']).dt.day
    dfo['DANIO'] = pd.to_datetime(dfo['FECHA']).dt.dayofyear
    dfo['PSEM'] = 1 + ((pd.to_datetime(dfo['FECHA']) - anio) // pd.Timedelta(days=7))
    dfo['DSEM'] = pd.to_datetime(dfo['FECHA']).dt.dayofweek
    dfo['ESDIA'] = [0 if (x==0 or x==3) else 1 for x in dfo['JORN']]
    dfo['NSEM'] = pd.to_datetime(dfo['FECHA']).dt.week
    dfo['FSEM'] = [0 if x<4 else 1 for x in dfo['DSEM']]
    return dfo
   

In [5]:
# filtrando los datos
dfp = data_crime(nuse1); dfp

,INDEX_TIME,LONGITUD,LATITUD,LOCALIDAD,COD_UPZ,UPZ,FECHA,MES,ANIO,JORN,DIA,DANIO,PSEM,DSEM,ESDIA,NSEM,FSEM
0,2197950,-74.083227,4.761108,SUBA,UPR1,UPR ZONA NORTE,2018-09-08 10:57:27,9,2018,1,8,251,245,5,1,36,1
1,988208,-74.083227,4.761108,SUBA,UPR1,UPR ZONA NORTE,2014-05-18 01:49:11,5,2014,0,18,138,20,6,0,20,1
2,2196412,-74.083227,4.761108,SUBA,UPR1,UPR ZONA NORTE,2019-10-21 12:20:30,10,2019,2,21,294,304,0,1,43,0
3,1610097,-74.083227,4.761108,SUBA,UPR1,UPR ZONA NORTE,2015-01-17 20:03:29,1,2015,3,17,17,55,5,0,3,1
4,11401,-74.083227,4.761108,SUBA,UPR1,UPR ZONA NORTE,2014-03-31 11:58:32,3,2014,1,31,90,14,0,1,14,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2245400,442468,-74.065031,4.634602,CHAPINERO,UPZ99,CHAPINERO,2015-10-13 02:25:38,10,2015,0,13,286,94,1,0,42,0
2245401,7051,-74.064442,4.639318,CHAPINERO,UPZ99,CHAPINERO,2014-02-22 11:38:34,2,2014,1,22,53,8,5,1,8,1
2245402,771523,-74.063171,4.648647,CHAPINERO,UPZ99,CHAPINERO,2015-01-21 21:07:22,1,2015,3,21,21,56,2,0,4,0
2245403,1179943,-74.063405,4.649713,CHAPINERO,UPZ99,CHAPINERO,2018-06-23 20:08:33,6,2018,3,23,174,234,5,0,25,1


#### 3. Generación del conjunto de datos.
> Las series de tiempo para la predicción del crimen podrán ser agrupadas y generadas para los años del 2014 a 2019. Las series de tiempo pueden darse en periodos:
* Menusales
* Semanales
* Diarios
* Para un día en específico de la semana (lunes, martes, miércoles, jueves, viernes)
* Para una jornada en específico y día de la semana en específico (martes en la madrugada)
* Para una jornada, todos los días (Lunes a Domingo en la tarde)
* Semanal entre semana (Lunes a Jueves)
* Semanal fines de semana (Viernes a Domingo)

##### 3.1 Funciones para generar el conjunto de datos.
> Existen dos funciones:
1. **data_series:** genera los datos para series de tiempo en los periodos anteriormente mencionados.
2. **st_series:** filtra los datos a partir de un punto espacial de interés y genera el conjunto de datos para la serie de tiempo dependiendo del periodo seleccionado.

La función **bw_intervals** encuentra los incidentes a lo largo del conjunto de datos que se encuentran dentro de una área circular alrededor del punto de interés.

##### 3.1.1 Función data_series

A continuación se especifican los parámetros de la función:

Parámetro | Descripción
:--------: | -------
fi | Año inicial para el conjunto de datos. Año de inicio de la serie de tiempo.
ff | Año final para el conjunto de datos. Año de fin de la serie de tiempo.
df | Dataframe filtardo de la función data_crime. 
tp | Tipo de serie de tiempo. Puede ser : mensual = 0, semanal = 1, diario = 2, día de la semana = 3, jornada (en un día de la semana) = 4, una jornada todos los días = 5, entre semana = 6, fines de semana = 7
ds | Día de la semana para la serie de tiempo. Permitido de 0 a 6, donde 0 es lunes y 6 es domingo.
jd | Jornada del día. Permitido de 0 a 3. 
 

In [6]:
def data_series(fi, ff, df, tp, ds, jd):
    
    dfi, dfo = None, None
    index, serie = list(), list()
    
    if tp >= 0 and tp <= 7:
        # mensual
        if tp == 0:
            # indice para todos los meses
            index = pd.date_range(start=fi, end=ff, freq='MS', 
                                  closed="left").strftime('%Y-%m').to_list()     
            # agrupando por
            serie = ['ANIO', 'MES']
            # preparando dataframe de salida
            data = {'ANIO': pd.to_datetime(index).year.tolist(), 
                    'MES': pd.to_datetime(index).month.tolist(),
                    'EVENTS': np.zeros(len(index), dtype=int)}
            dfo = pd.DataFrame(index=index, data=data)
            
            # agrupación por año y mes
            dfi = pd.DataFrame(df.groupby(serie)['UPZ'].count()).reset_index()
            dfi['FECHA'] = pd.to_datetime(dfi['ANIO'].astype(str) + 
                                          '-' + dfi['MES'].astype(str)).dt.strftime('%Y-%m')
            
            for i in range(len(dfi)):
                if dfo.index[i] == dfi['FECHA'].iloc[i]:
                    dfo['EVENTS'].iloc[i] = dfi['UPZ'].iloc[i]
            return dfo
        # semanal
        if tp == 1:
            # indice para todas las semanas
            index = pd.date_range(start=fi, end=ff, freq='W', 
                                  closed="left").strftime('%Y-%m-%d')
            indexb = pd.date_range(start=fi, end=ff, freq='W', 
                                  closed="left").strftime('%Y-%m-%d')
            indexb = (pd.to_datetime(index) + pd.offsets.Week(weekday=0)).shift(-1, freq='W-MON')
            
            # or sirve
            #pd.date_range(start='2013-12-29', end='2020', 
            #              freq='W-MON', closed="left").strftime('%Y-%m-%d')
            # agrupando por
            serie = ['PSEM']
            data = {'ANIO': pd.to_datetime(indexb).year.tolist(),
                    'NSEM': np.arange(1, len(indexb) + 1),
                    'EVENTS': np.zeros(len(indexb), dtype=int)}
            # preparando dataframe de salida
            #dfo = pd.DataFrame(index=index.to_list(), data=data)
            dfo = pd.DataFrame(index=indexb, data=data)
            dfi = pd.DataFrame(df.groupby(serie)['UPZ'].count()).reset_index() 
            dfi = dfi.sort_values('PSEM')
            
            for i in range(len(dfo)):
                
                d = dfi[dfi['PSEM'] == dfo['NSEM'].iloc[i]]
                if d.empty == False:
                    dfo['EVENTS'].iloc[i] = int(d['UPZ'])

            return dfo
        # diario
        if tp == 2:
            # indice para todos los días
            index = pd.date_range(start=fi, end=ff, freq='D', 
                                 closed="left").to_list()
            # agrupando por
            serie = ['ANIO', 'MES', 'DIA']
            # preparando dataframe de salida
            data = {'ANIO': pd.to_datetime(index).year.tolist(), 
                    'MES': pd.to_datetime(index).month.tolist(),
                    'DIA': pd.to_datetime(index).day.tolist(),
                    'EVENTS': np.zeros(len(index), dtype=int)} 
            dfo = pd.DataFrame(index=index, data=data)
            # generando agrupamiento
            dfi = pd.DataFrame(df.groupby(serie)['UPZ'].count()).reset_index()    
            dfi['FECHA'] = pd.to_datetime(dfi['ANIO'].astype(str) + 
                                          '-' + dfi['MES'].astype(str) + 
                                          '-' + dfi['DIA'].astype(str))
            for i in range(len(dfo)):
                d = dfi[(dfi['ANIO'] == dfo['ANIO'].iloc[i]) & 
                            (dfi['MES'] == dfo['MES'].iloc[i]) & 
                            (dfi['DIA'] == dfo['DIA'].iloc[i])]
                if d.empty == False:
                    dfo['EVENTS'].iloc[i] = int(d['UPZ'])
            return dfo
        # día de la semana
        if tp == 3 and ds != None:
            # lunes 0 - domingo 6
            if ds >= 0 and ds <= 6:
                # índice de todos los días
                days = ['W-MON', 'W-TUE', 'W-WED', 'W-THU', 'W-FRI', 'W-SAT', 'W-SUN']
                index = pd.date_range(start=fi, end=ff, freq=days[ds], 
                                     closed="left")
                # índice con solo un día en específico (l, m, mi, j, v, s, d)
                #index = index[pd.to_datetime(index).dayofweek == ds].to_list()
                # agrupando por
                serie = ['ANIO','MES','NSEM', 'DSEM']
                # preparando dataframe de salida
                data = {'ANIO': pd.to_datetime(index).year.tolist(), 
                    'MES': pd.to_datetime(index).month.tolist(),
                    'NSEM': pd.to_datetime(index).week.tolist(),
                    'DSEM': pd.to_datetime(index).dayofweek.tolist(),
                    'EVENTS': np.zeros(len(index), dtype=int)}     
                dfo = pd.DataFrame(index=index, data=data)
                # generando agrupamiento
                dfi = pd.DataFrame(df.groupby(serie)['UPZ'].count()).reset_index()
                dfi = dfi[dfi['DSEM'] == ds]
                
                for i in range(len(dfo)):
                    d = dfi[(dfi['ANIO'] == (dfo['ANIO'].iloc[i])) & 
                            (dfi['MES'] == dfo['MES'].iloc[i]) & 
                            (dfi['NSEM'] == dfo['NSEM'].iloc[i]) &
                           (dfi['DSEM'] == dfo['DSEM'].iloc[i])]
                    
                    if d.empty == False:
                        dfo['EVENTS'].iloc[i] = int(d['UPZ'])
                        
                return dfo
        
        # día de la semana y jornada
        if tp == 4 and ds != None and jd != None:
            
            if (ds >= 0 and ds <= 6) and (jd >= 0 and jd <= 3):
                
                index = pd.date_range(start=fi, end=ff, freq='D', 
                                     closed="left").strftime('%Y-%m-%d')
                # índice con solo un día en específico (l, m, mi, j, v, s, d)
                index = index[pd.to_datetime(index).dayofweek == ds].to_list()
                # agrupando por
                serie =  ['ANIO','MES','NSEM', 'DSEM', 'JORN']
                # preparando dataframe de salida
                data = {'ANIO': pd.to_datetime(index).year.tolist(), 
                    'MES': pd.to_datetime(index).month.tolist(),
                    'NSEM': pd.to_datetime(index).week.tolist(),
                    'DSEM': pd.to_datetime(index).dayofweek.tolist(),
                    'JORN': np.full(len(index), jd, dtype=int),   
                    'EVENTS': np.zeros(len(index), dtype=int)}     
                dfo = pd.DataFrame(index=index, data=data)
                # generando agrupamiento
                dfi = pd.DataFrame(df.groupby(serie)['UPZ'].count()).reset_index()
                dfi = dfi[(dfi['DSEM'] == ds) & (dfi['JORN'] == jd)]
                
                for i in range(len(dfi)):
                    if (dfo['ANIO'].iloc[i] == dfi['ANIO'].iloc[i]) and (dfo['MES'].iloc[i] == dfi['MES'].iloc[i]) and (dfo['NSEM'].iloc[i] == dfi['NSEM'].iloc[i]) and (dfo['DSEM'].iloc[i] == dfi['DSEM'].iloc[i]) and (dfo['JORN'].iloc[i] == dfi['JORN'].iloc[i]): 
                            dfo['EVENTS'].iloc[i] = dfi['UPZ'].iloc[i]
                return dfo
        # todas las jornadas todos los días
        if tp == 5 and jd != None:
            
            if jd >= 0 and jd <= 3:
                # índice para todos los días cada 6 horas 
                index = pd.date_range(start=fi, end=ff, freq='6H',
                                      closed="left").strftime('%Y-%m-%d %H').to_list()
                # agrupando por
                serie = ['ANIO','MES','NSEM', 'DSEM', 'JORN']
                # preparando dataframe de salida
                data = {'ANIO': pd.to_datetime(index).year.tolist(), 
                    'MES': pd.to_datetime(index).month.tolist(),
                    'NSEM': pd.to_datetime(index).week.tolist(),
                    'DSEM': pd.to_datetime(index).dayofweek.tolist(),
                    'JORN': pd.to_datetime(index).hour//6,   
                    'EVENTS': np.zeros(len(index), dtype=int)}     
                dfo = pd.DataFrame(index=index, data=data)
                # generando agrupamiento
                dfi = pd.DataFrame(df.groupby(serie)['UPZ'].count()).reset_index()
                
                for i in range(len(dfi)):
                    if (dfo['ANIO'].iloc[i] == dfi['ANIO'].iloc[i]) and (dfo['MES'].iloc[i] == dfi['MES'].iloc[i]) and (dfo['NSEM'].iloc[i] == dfi['NSEM'].iloc[i]) and (dfo['DSEM'].iloc[i] == dfi['DSEM'].iloc[i]) and (dfo['JORN'].iloc[i] == dfi['JORN'].iloc[i]): 
                            dfo['EVENTS'].iloc[i] = dfi['UPZ'].iloc[i]
                return dfo
        # semanas de lunes a jueves
        if tp == 6:
            # índice para semanas de lunes a jueves          
            index = pd.date_range(start=fi, end=ff, freq='W-THU',
                                      closed="left").strftime('%Y-%m-%d').to_list()
            # agrupando por
            serie = ['ANIO','NSEM']
            # preparando dataframe de salida
            data = {'ANIO': pd.to_datetime(index).year.tolist(), 
                    'NSEM': pd.to_datetime(index).week.tolist(),
                    'EVENTS': np.zeros(len(index), dtype=int)}     
            dfo = pd.DataFrame(index=index, data=data)
            # filtro días de lunes a jueves
            dfi = df[df['DSEM'] <= 3]
             # generando agrupamiento
            dfi = pd.DataFrame(dfi.groupby(serie)['UPZ'].count()).reset_index()
            for i in range(len(dfi)):
                if dfo['ANIO'].iloc[i] == dfi['ANIO'].iloc[i] and dfo['NSEM'].iloc[i] == dfi['NSEM'].iloc[i]:
                    dfo['EVENTS'].iloc[i] = dfi['UPZ'].iloc[i]
            return dfo
        if tp == 7:
            # índice para semanas de viernes a domingo
            index = pd.date_range(start=fi, end=ff, freq='D',
                                      closed="left").strftime('%Y-%m-%d')
            index = index[pd.to_datetime(index).dayofweek == 6]
            # agrupando por
            serie = ['ANIO','NSEM']
            # preparando dataframe de salida
            data = {'ANIO': pd.to_datetime(index).year.tolist(), 
                    'NSEM': pd.to_datetime(index).week.tolist(),
                    'EVENTS': np.zeros(len(index), dtype=int)}     
            dfo = pd.DataFrame(index=index.to_list(), data=data)
            # filtro días de lunes a jueves
            dfi = df[(df['DSEM'] > 3) & (df['DSEM'] <= 6)]
             # generando agrupamiento
            dfi = pd.DataFrame(dfi.groupby(serie)['UPZ'].count()).reset_index()
            for i in range(len(dfi)):
                if dfo['ANIO'].iloc[i] == dfi['ANIO'].iloc[i] and dfo['NSEM'].iloc[i] == dfi['NSEM'].iloc[i]:
                    dfo['EVENTS'].iloc[i] = dfi['UPZ'].iloc[i]
            return dfo
            

In [7]:
# encontra intervalos ancho de banda
def bw_intervals(mts, long, lat):
    
    lt_min, lt_max, lg_min, lg_max = 0, 0, 0, 0
    
    cearth = 40075           # earth circumference in kmss
    onedeg_km = cearth / 360 # 1 degree to kms 
    mt_km = mts / 1000       # mts to kms
    
    deglt = mt_km / onedeg_km              # convert kms to deglat
    deglg = mt_km / m.cos(m.radians(lat))  # convert kms to deglong
        
    if lat < 0:
        
        lt_min = -deglt + lat 
        lt_max = lat + deglt
    else:
        
        lt_min = lat - deglt   
        lt_max = lat + deglt
    
    if long < 0:
        
        lg_min = -deglg + long
        lg_max = long + deglg
    else:
        
        lg_min = long - deglt   
        lg_max = long + deglt
    
    return [lg_min, lg_max], [lt_min, lt_max] 

In [8]:
# Tipos de series de tiempo
# mensual = 0, semanal = 1, diario = 2, dia semana = 3, jornada (dia de la semana) = 4
# jornada todos los dias = 5, entre semana = 6, fines de semana = 7

# parámetros: fi: fecha de inicio, ff: fecha final, df: dataframe, 
#             tp: tipo de serie, ds: día de la semana,
#             jd: jornada en un día específico, mts: ancho de banda en metros
#             plg: valor longitud punto de interés, 
#             plt: valor de latitud punto de interés.
def st_series(fi, ff, df, tp, ds, jd, mts, plg, plt):
    
    # encontrando intervalos de ancho de banda
    bwlong, bwlat = bw_intervals(mts, plg, plt)
    # df.rename(columns={'Unnamed: 0': 'COUNT'}, inplace=True)
    # dataframe filtrado por latitud y longitud
    dff = df[((df['LATITUD'] >= bwlat[0]) & (df['LATITUD'] <= bwlat[1])) & 
             ((df['LONGITUD'] >= bwlong[0]) & (df['LONGITUD'] <= bwlong[1]))]

    # dataframe para la serie de tiempo de acuerdo al tipo
    # parámetros: df: dataframe, tp: tipo de serie, a: año, ds: día de la semana,
    #             jd: jornada en un día específico
    dffs = data_series(fi, ff, dff, tp, ds, jd)
    # return pd.DataFrame(columns=['COUNT'], data=dffg)
    return dffs

#### 4. Predicción

In [23]:
# parámetros
# fid: fecha inicial del total de conjunto de datos
# ffd: fecha final del conjunto de datos
# fie: fecha inicial del cojunto de datos de entrenamiento
# ffe: fecha final del conjunto de datos de entrenamiento
# dfn: dataframe de nuse filtrado
# bwt: ancho de banda en metros
# lon: coordenada longitudinal del punto de interés
# lat: coordenada latitudinal del punto de interés
def pred_mt (fid, ffd, fie, ffe, nsem, dfn, bwt, lon, lat):
    
    fli = pd.to_datetime('2014') # fecha límite inicial
    flf = pd.to_datetime('2020') # fecha límite final
    fid = pd.to_datetime(fid) 
    ffd = pd.to_datetime(ffd) 
    fie = pd.to_datetime(fie)
    ffe = pd.to_datetime(ffe) 
    pred = False
    
    if (fid >= fli and ffd < flf) and (fie >= fid and ffe <= ffd):
        pred = True
    
    if pred:
        # ajustando las fechas para los datos
        if pd.to_datetime(fid).dayofweek != 0:
            dias = pd.to_datetime(fid).dayofweek
            fid = pd.to_datetime(fid) - pd.DateOffset(days=dias) 
            
        if pd.to_datetime(ffd).dayofweek != 0:
            dias = pd.to_datetime(ffd).dayofweek
            ffd = pd.to_datetime(ffd) - pd.DateOffset(days=dias) 
            
        if pd.to_datetime(fie).dayofweek != 0:
            dias = pd.to_datetime(fie).dayofweek
            fie = pd.to_datetime(fie) - pd.DateOffset(days=dias) 
        
        if pd.to_datetime(ffe).dayofweek != 0:
            dias = pd.to_datetime(ffe).dayofweek
            ffe = pd.to_datetime(ffe) - pd.DateOffset(days=dias)
        
        d_sem = st_series('2014', '2020', dfn, 1, 0, 0, bwt, lon, lat)
        d_tots = d_sem[fid:ffd] # dataframe base total de eventos semana
        d_ents = d_sem[fie:ffe] # dataframe base para entrenamiento semana
        
        d_dia = st_series('2014', '2020', dfn, 2, 0, 0, bwt, lon, lat)
        d_totd = d_dia[fid:ffd] # dataframe base total de eventos diario
        d_entd = d_dia[fie:ffe] # dataframe base para entrenamiento diario
        
        d_comp = pd.DataFrame()
        for i in range(7):
            # df de composición parcial por día
            dcp = st_series('2014', '2020', dfn, 3, i, 0, bwt, lon, lat)
            d_comp = d_comp.append(dcp)
        
        # df descomposición de la serie de tiempo semanal
        dfds = pd.DataFrame(d_ents['EVENTS'])
        decs = decompose(dfds, period=52)
        
        # df descomposición de la serie de tiempo diaria
        dfdd = pd.DataFrame(d_entd['EVENTS'])
        decd = decompose(dfdd, period=28)
        
        # fecha final e inicial de predicción semanal
        ffps = pd.to_datetime(ffe) + pd.Timedelta(nsem, 'W')
        fips = pd.to_datetime(ffps) - pd.Timedelta(nsem - 1, 'W')
        s = d_sem[fie:ffps]
        # fecha final e inicial de predicción diaria
        ffpd = pd.to_datetime(ffe) + pd.Timedelta(nsem*7, 'D')
        fipd = pd.to_datetime(ffpd) - pd.Timedelta((nsem*7) - 1, 'D')
        
        d_totsp = d_tots
        d_totdp = d_totd
        # configurando extensión de fechas para predicción cuando la
        # ventana de predicción excede el conjunto de datos
        
        # predicción tendencia y estacional para las semanas
        pts = forecast(decs, steps=nsem, fc_func=drift)
        pes = forecast(decs, steps=nsem, fc_func=drift, seasonal=True)

        if ffps <= ffd:
            rms1 = np.sum((pes.iloc[:,0].values - d_totsp['EVENTS'][fips:ffps].values)**2)**0.5/len(pes)
            err1 = 100*np.sum(abs(pes.iloc[:,0].values - d_totsp['EVENTS'][fips:ffps].values)/pts['drift'].values)/len(pes)
            SSE = np.sum((pes.iloc[:,0].values - d_totsp['EVENTS'][fips:ffps].values.mean())**2)
            SSR = np.sum((pes.iloc[:,0].values - d_totsp['EVENTS'][fips:ffps].values)**2)
            ve1 = SSE/SSR
        elif fips < ffd and ffps > ffd:
            rms1 = np.sum((pes.iloc[:,0].values - d_totsp['EVENTS'][fips:ffd].values)**2)**0.5/len(pes)
            err1 = 100*np.sum(abs(pes.iloc[:,0].values - d_totsp['EVENTS'][fips:ffd].values)/pts['drift'].values)/len(pes)   
            SSE = np.sum((pes.iloc[:,0].values - d_totsp['EVENTS'][fips:ffd].values.mean())**2)
            SSR = np.sum((pes.iloc[:,0].values - d_totsp['EVENTS'][fips:ffd].values)**2)
            ve1 = SSE/SSR
        else:
            rms1, err1, ve1 = 0, 0, 0 
            
        des1 = np.std(decs.resid.EVENTS) / np.mean(d_ents.EVENTS)
        
        # predicción tendencia y estacional para todos los días
        ptd = forecast(decd, steps=nsem*7, fc_func=drift)
        ped = forecast(decd, steps=nsem*7, fc_func=drift, seasonal=True)
        
        if ffpd <= ffd:
            rms2 = np.sum((ped.iloc[:,0].values - d_totdp['EVENTS'][fipd:ffpd].values)**2)**0.5/len(ped)
            err2 = 100*np.sum(abs(ped.iloc[:,0].values - d_totdp['EVENTS'][fipd:ffpd].values)/ptd['drift'].values)/len(ped)
            SSE = np.sum((ped.iloc[:,0].values - d_totdp['EVENTS'][fipd:ffpd].values.mean())**2)
            SSR = np.sum((ped.iloc[:,0].values - d_totdp['EVENTS'][fipd:ffpd].values)**2)
            ve2 = SSE/SSR
        elif fipd < ffd and ffpd > ffd:
            rms2 = np.sum((ped.iloc[:,0].values - d_totdp['EVENTS'][fipd:ffd].values)**2)**0.5/len(ped)
            err2 = 100*np.sum(abs(ped.iloc[:,0].values - d_totdp['EVENTS'][fipd:ffd].values)/ptd['drift'].values)/len(ped)        
            SSE = np.sum((ped.iloc[:,0].values - d_totdp['EVENTS'][fipd:ffd].values.mean())**2)
            SSR = np.sum((ped.iloc[:,0].values - d_totdp['EVENTS'][fipd:ffd].values)**2)
            ve2 = SSE/SSR
        else:
            rms2, err2, ve2 = 0, 0, 0
            
        des2 = np.std(decd.resid.EVENTS) / np.mean(d_entd.EVENTS)    
        # gráficos de los resultados   
        
        fig1 = go.Figure()
        fig1.layout.template = 'plotly_dark'
        # serie de tiempo de los datos originales semanales
        fig1.add_trace(go.Scatter(x=d_totsp.index, y=d_tots.EVENTS, mode='lines', name='Origin',line=dict(color='#8aff8c')))
        
        # gráficos descomposición estacional, residuo y tendencia semanales
        fig1.add_trace(go.Scatter(x=d_ents.index, y=decs.seasonal.EVENTS, mode='lines', name='Season'))
        fig1.add_trace(go.Scatter(x=d_ents.index, y=decs.resid.EVENTS, mode='lines', name='Resid'))
        fig1.add_trace(go.Scatter(x=[ffe, fips], y=[decs.trend.EVENTS[len(decs.trend.EVENTS) - 1], pts.drift[0]], mode='lines', line = dict(color='red'), showlegend = False))
        fig1.add_trace(go.Scatter(x=d_ents.index, y=decs.trend.EVENTS, mode='lines', name='Trend'))
        
        # gráfico predicción tendencia semanal
        fig1.add_trace(go.Scatter(x=pts.index, y=pts.drift, mode='lines+markers', name='Trend-fcast', line = dict(color='red')))
        # gráfico predicción estacional semanal
        fig1.add_trace(go.Scatter(x=pes.index, y=pes['drift+seasonal'], mode='lines+markers', name='Predict', line = dict(color='#8accff', width=1, dash='dot')))
        fig1.update_layout(title = 'Predicción semenal: rms:%1.2f   std:%1.2f   err:%1.2f   ve:%1.2f'%(rms1, des1, err1, ve1))
        #fig1.update_layout(title = 'Predicción semenal: rms:%1.2f   std:%1.2f   err:%1.2f   ve:%1.2f'%(rms1, des1, err1, ve1), width=1000,height=400)
        fig1.show()
        
         # serie de tiempo de los datos originales diaria
        fig2 = go.Figure()
        fig2.layout.template = 'plotly_dark'

        fig2.add_trace(go.Scatter(x=d_totdp.index, y=d_totd.EVENTS, mode='lines', name='Origin', line=dict(color='#8aff8c')))
        
        # gráficos descomposición estacional, residuo y tendencia diaria
        fig2.add_trace(go.Scatter(x=d_entd.index, y=decd.seasonal.EVENTS, mode='lines', name='Season'))
        fig2.add_trace(go.Scatter(x=d_entd.index, y=decd.resid.EVENTS, mode='lines', name='Resid'))
        fig2.add_trace(go.Scatter(x=[ffe, fipd], y=[decd.trend.EVENTS[len(decd.trend.EVENTS) - 1], ptd.drift[0]], mode='lines', line = dict(color='red'), showlegend = False))
        fig2.add_trace(go.Scatter(x=d_entd.index, y=decd.trend.EVENTS, mode='lines', name='Trend'))
        
        # gráfico predicción tendencia diaria
        fig2.add_trace(go.Scatter(x=ptd.index, y=ptd.drift, mode='lines+markers', name='Trend-fcast', line = dict(color='red')))
        # gráfico predicción estacional diaria
        fig2.add_trace(go.Scatter(x=ped.index, y=ped['drift+seasonal'], mode='lines+markers', name='Predict', line = dict(color='#8accff', width=1, dash='dot')))
        
        fig2.update_layout(title = 'Predicción diaria rms:%1.2f   std:%1.2f   err:%1.2f   ve:%1.2f'%(rms2, des2, err2, ve2))
        #fig2.update_layout(title = 'Predicción diaria rms:%1.2f   std:%1.2f   err:%1.2f   ve:%1.2f'%(rms2, des2, err2, ve2), width=1000,height=400)
        fig2.update_yaxes(range=[0, 1000])
        fig2.update_xaxes(autorange= False, range=[pd.to_datetime(ffe) - pd.Timedelta(10, 'D'), ffpd])
        fig2.show()
        
        # datos para serie de tiempo en días en específico
        days = ['W-MON', 'W-TUE', 'W-WED', 'W-THU', 'W-FRI', 'W-SAT', 'W-SUN']
        dayse = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']
        
        dpt = pd.DataFrame() # df de predicción acumulada en tendencia
        dpe = pd.DataFrame() # df de predicción acumulada estacional
        dtt = pd.DataFrame() # df total de los datos
        dte = pd.DataFrame() # df entrenamiento acumulado
        dec = pd.DataFrame() # df de descomposición acumulada
        for i in range(7):
            
            dias = pd.to_datetime(ffd).dayofweek
            ffd = pd.to_datetime(ffd) + pd.DateOffset(days=abs(i - dias)) 
                   
            dias = pd.to_datetime(ffe).dayofweek
            ffe = pd.to_datetime(ffe) + pd.DateOffset(days=abs(i - dias))
            
            d_de = st_series('2014', '2020', dfn, 3, i, 0, bwt, lon, lat)
            d_totde = d_de[fid:ffd] # dataframe base total de eventos día específico
            d_entde = d_de[fie:ffe] # dataframe base para entrenamiento día específico
            
            # df descomposición de la serie de tiempo para un día en específico
            # (l, m, mi, j, v, s, d)
            dfdde = pd.DataFrame(d_entde['EVENTS'])
            decde = decompose(dfdde, period=7)

            # fecha final e inicial de predicción día en específico
            fipde = pd.to_datetime(ffe) + pd.Timedelta(1, 'W')
            ffpde = pd.to_datetime(fipde) + pd.Timedelta(nsem -1, 'W')
            
            d_totdep = d_totde 
            
            # predicción tendencia y estacional para las semanas
            ptde = forecast(decde, steps=nsem, fc_func=drift)
            pede = forecast(decde, steps=nsem, fc_func=drift, seasonal=True)
            
            if ffpd <= ffd:
                rms3 = np.sum((pede.iloc[:,0].values - d_totdep['EVENTS'][fipde:ffpde].values)**2)**0.5/len(ped)
                err3 = 100*np.sum(abs(pede.iloc[:,0].values - d_totdep['EVENTS'][fipde:ffpde].values)/ptde['drift'].values)/len(pede)
                SSE = np.sum((pede.iloc[:,0].values - d_totdep['EVENTS'][fipde:ffpde].values.mean())**2)
                SSR = np.sum((pede.iloc[:,0].values - d_totdep['EVENTS'][fipde:ffpde].values)**2)
                ve3 = SSE/SSR
            elif fipde < ffd and ffpde > ffd:
                rms3 = np.sum((pede.iloc[:,0].values - d_totdep['EVENTS'][fipde:ffd].values)**2)**0.5/len(ped)
                err3 = 100*np.sum(abs(pede.iloc[:,0].values - d_totdep['EVENTS'][fipde:ffd].values)/ptde['drift'].values)/len(pede)        
                SSE = np.sum((pede.iloc[:,0].values - d_totdep['EVENTS'][fipde:ffd].values.mean())**2)
                SSR = np.sum((pede.iloc[:,0].values - d_totdep['EVENTS'][fipde:ffd].values)**2)
                ve3 = SSE/SSR
            else:
                rms3, err3, ve3 = 0, 0, 0
            
            des3 = np.std(decde.resid.EVENTS) / np.mean(d_entde.EVENTS)
            
            dpt = dpt.append(ptde); dpe = dpe.append(pede); dtt = dtt.append(d_totdep); dte = dte.append(d_entde);
            dect = pd.DataFrame({'seasonal': decde.seasonal.EVENTS, 'resid': decde.resid.EVENTS, 'trend': decde.trend.EVENTS}, index=dfdde.index)
            dec = dec.append(dect); 
            
        # organización de los datos de predicción (1lunes, 1martes, 1miércoles, 1jueves.....2lunes, 2martes....) 
        dpt = dpt.sort_index(); dpe = dpe.sort_index(); dtt = dtt.sort_index(); dte = dte.sort_index(); dec = dec.sort_index();
        # gráficos de los resultados    
        fig3 = go.Figure()
        fig3.layout.template = 'plotly_dark'
        # serie de tiempo de los datos originales día a día de la semana y compuesto
        fig3.add_trace(go.Scatter(x=dtt.index, y=dtt.EVENTS, mode='lines', name='Origin', line=dict(color='#8aff8c')))

         # gráficos descomposición estacional, residuo y tendencia diarias día en específico
        fig3.add_trace(go.Scatter(x=dec.index, y=dec.seasonal, mode='lines', name='Season'))
        fig3.add_trace(go.Scatter(x=dec.index, y=dec.resid, mode='lines', name='Resid'))
        fig3.add_trace(go.Scatter(x=[ffe, dpt.index[0]], y=[dec.trend[len(dec.trend) - 1], dpt.drift[0]], mode='lines', line = dict(color='red'), showlegend = False))
        fig3.add_trace(go.Scatter(x=dec.index, y=dec.trend, mode='lines', name='Trend'))

        # gráfico predicción tendencia semanal
        fig3.add_trace(go.Scatter(x=dpt[fipd:ffpd].index, y=dpt.drift, mode='lines+markers', name='Trend-fcast', line = dict(color='red')))
        # gráfico predicción estacional semanal
        fig3.add_trace(go.Scatter(x=dpe[fipd:ffpd].index, y=dpe['drift+seasonal'], mode='lines+markers', name='Predict', line = dict(color='#8accff', width=0.5, dash='dot')))
        fig3.update_layout(title = 'Predicción por día acumulada  rms:%1.2f   std:%1.2f   err:%1.2f    ve:%1.2f'%(rms3, des3, err3, ve3))
        #fig3.update_layout(title = 'Predicción por día acumulada  rms:%1.2f   std:%1.2f   err:%1.2f    ve:%1.2f'%(rms3, des3, err3, ve3), width=1000,height=400)
        fig3.update_yaxes(range=[0, 1000])
        fig3.update_yaxes(automargin=True)
        fig3.show()
        
    return d_tots, pts, pes
    

In [26]:
# resultados para mostrar
dtot, pten, pest = pred_mt('2015-01-22', '2019-11-24', '2016-01-22', '2019-07-22', 10, dfp, 500, -74.185244, 4.60905)


In [ ]:
dtot, pten, pest = pred_mt('2015-01-22', '2019-11-24', '2016-01-22', '2019-11-24', 10, dfp, 2000, -74.185244, 4.60905)